# Merging table to create Dataset for feature engineering

In [1]:
import pandas as pd
import requests
import json
import time
import numpy as np

pd.set_option('display.max_columns', None)

In [2]:
# Data import
games = pd.read_csv("../raw_data/games_w_venue.csv")
players = pd.read_csv("../raw_data/players.csv")
stadiums = pd.read_csv("../raw_data/stadiums.csv")
teams = pd.read_csv("../raw_data/teams.csv")
data = pd.read_csv("../raw_data/all_ab_raw_data_w_target.csv")
added_data = pd.read_csv("../raw_data/all_ab_raw_data_add_columns.csv")

In [3]:
data.shape

(143088, 16)

In [4]:
# Merging data and games
games = games.rename(columns={"id": "game_id"})
data = data.merge(games, how="left", on='game_id')
data.shape

(143088, 35)

In [5]:
# Merging data and hitters
hitters = players[~players.id.duplicated(keep="first")]
hitters = hitters.add_prefix("hitter_")
data = data.merge(hitters, how="left", on="hitter_id")
data.shape

(143088, 45)

In [6]:
# Merging data and pitchers
pitchers = players[~players.id.duplicated(keep="first")]
pitchers = pitchers.add_prefix("pitcher_")
data = data.merge(pitchers, how="left", on="pitcher_id")
data.shape

(143088, 55)

In [7]:
# Merging data and home team
home_team = teams.add_prefix("home_team_")
data = data.rename(columns={"home_team": "home_team_id"})
data = data.merge(home_team, how="left", on="home_team_id")
data.shape

(143088, 58)

In [8]:
# Merging data and away team
away_team = teams.add_prefix("away_team_")
data = data.rename(columns={"away_team": "away_team_id"})
data = data.merge(away_team, how="left", on="away_team_id")
data.shape

(143088, 61)

In [9]:
# Merging data and stadium
venue = stadiums.add_prefix("stadium_")
data = data.rename(columns={"venue_id": "stadium_id"})
data = data.merge(venue, how="left", on="stadium_id")
data.shape

(143088, 75)

In [10]:
# Merging data and away adress
away_stadium = stadiums
away_stadium["abbr"] = teams.abbr
away_stadium = away_stadium.add_prefix("away_stadium_")
away_stadium = away_stadium.rename(columns={"away_stadium_abbr": "away_team_abbr"})
data = data.merge(away_stadium, how="left", on="away_team_abbr")
data.shape

(143088, 90)

In [11]:
added_data

,Unnamed: 0,id,at_bat_end_time,pitch_type_code,pitch_type_des,pitch_speed_mph,pitch_count_at_bat,pitcher_pitch_count_at_bat_start,outs_at_start,output_code
0,0,67cfd85d-029a-4e5d-9ad3-cf7c1f29c303,2023-05-08T23:43:05+00:00,FF,Four-Seam Fastball,96.5,4.0,0.0,0.0,oFO
1,0,1ec4235d-7ee4-47f7-a262-40ef0a76acd2,2023-05-08T23:45:01+00:00,CU,Curveball,76.3,5.0,4.0,1.0,oLO
2,0,d5d817fa-a490-45b3-9884-0e6a70d97e3e,2023-05-08T23:45:36+00:00,FF,Four-Seam Fastball,94.8,1.0,9.0,2.0,oGO
3,0,c4cfeef1-8fa9-4ae4-aa35-c56dabe5bde4,2023-05-08T23:47:55+00:00,FF,Four-Seam Fastball,91.8,1.0,0.0,0.0,oFO
4,0,f8337219-44fc-4459-afb2-02778bd9ee4c,2023-05-08T23:50:07+00:00,FS,Splitter,83.3,6.0,1.0,1.0,oFO
...,...,...,...,...,...,...,...,...,...,...
143419,0,4c9a3a49-0e37-4065-8ecc-19ad6129cd1d,2023-04-14T01:53:00+00:00,FF,Four-Seam Fastball,93.1,10.0,9.0,1.0,oFO
143420,0,4d2038eb-6063-4d81-8f2a-fdd051a0a291,2023-04-14T01:54:52+00:00,FF,Four-Seam Fastball,92.1,4.0,19.0,2.0,kFT
143421,0,c48d6034-c7a9-42bf-8932-67e9947cf311,2023-04-14T01:58:37+00:00,CU,Curveball,87.0,3.0,0.0,0.0,oGO
143422,0,c96912db-3e1d-4db2-8651-856ef857bda4,2023-04-14T01:59:41+00:00,SI,Sinker,96.6,3.0,3.0,1.0,oGO


In [13]:
# Merging data and added parsing columns from JSON files
data = data.merge(added_data, how="left", left_index=True, right_index=True)
data.shape

(143088, 100)

In [14]:
data.describe()

,inning,temp_f,humidity,wind_speed_mph,mc_target,y_target,Unnamed: 0_x,game_number,attendance,reference,hitter_jersey_number,hitter_depth,pitcher_jersey_number,pitcher_depth,stadium_capacity,stadium_lat,stadium_lon,away_stadium_capacity,away_stadium_lat,away_stadium_lon,Unnamed: 0_y,pitch_speed_mph,pitch_count_at_bat,pitcher_pitch_count_at_bat_start,outs_at_start
count,143088.000000,143013.000000,143013.000000,143013.000000,143088.000000,143088.000000,143088.000000,143088.000000,142641.000000,143088.000000,111803.000000,111803.000000,95095.000000,95095.000000,142698.000000,142698.000000,142698.000000,143013.000000,143013.000000,143013.000000,143088.0,142320.000000,143041.000000,142935.000000,141423.000000
mean,4.958732,73.285184,54.075483,11.183962,0.466384,0.318986,945.036083,1.014732,29305.099880,717835.516032,21.444925,1.619518,44.552258,3.049887,42387.621004,38.127296,-92.610975,42391.641389,38.219927,-92.418223,0.0,89.058453,3.886054,29.438129,0.981135
std,2.583891,12.717081,19.632913,221.291345,0.860947,0.466085,545.990683,0.120479,11307.751334,546.710633,17.989597,0.964939,18.770786,1.723717,5475.053405,5.018305,16.405014,5489.398744,5.033629,16.425165,0.0,6.105643,1.897569,27.066052,0.816056
min,1.000000,34.000000,4.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,716887.000000,0.000000,1.000000,1.000000,1.000000,25025.000000,25.778057,-122.390621,25025.000000,25.778057,-122.390621,0.0,33.700000,0.000000,-6.000000,0.000000
25%,3.000000,65.000000,40.000000,4.000000,0.000000,0.000000,475.000000,1.000000,20286.000000,717363.000000,8.000000,1.000000,32.000000,2.000000,40000.000000,33.890672,-104.993349,40000.000000,33.890672,-104.993349,0.0,84.600000,2.000000,7.000000,0.000000
50%,5.000000,74.000000,56.000000,8.000000,0.000000,0.000000,942.000000,1.000000,30578.000000,717834.000000,18.000000,1.000000,45.000000,3.000000,41376.000000,39.097736,-87.656054,41700.000000,39.283787,-87.634833,0.0,89.800000,4.000000,20.000000,1.000000
75%,7.000000,82.000000,69.000000,12.000000,1.000000,1.000000,1416.000000,1.000000,38605.000000,718309.000000,28.000000,2.000000,57.000000,4.000000,45971.000000,41.830066,-80.006409,45971.000000,41.830066,-80.006409,0.0,93.900000,5.000000,50.000000,2.000000
max,14.000000,117.000000,100.000000,22369.000000,4.000000,1.000000,2430.000000,2.000000,55565.000000,718782.000000,99.000000,8.000000,99.000000,8.000000,56000.000000,47.589904,-71.098782,56000.000000,47.589904,-71.098782,0.0,104.800000,16.000000,117.000000,3.000000


In [15]:
data.duplicated().sum()

0

In [22]:
columns_to_remove_fp = list(('description', 'play_outcome', 'mc_target', 
                            'Unnamed: 0_x', 'status', 'coverage', 'game_number', 
                            'duration', 'double_header', 'entry_mode', 'reference', 
                            'venue', 'home', 'away', 'broadcast', 'rescheduled', 'hitter_status', 
                            'hitter_position', 'hitter_first_name', 'hitter_last_name', 'hitter_preferred_name',
                            'hitter_jersey_number', 'hitter_depth', 'hitter_team_id', 'hitter_team_nickname', 'pitcher_status',
                            'pitcher_position', 'pitcher_first_name', 'pitcher_last_name', 'pitcher_preferred_name', 'pitcher_jersey_number',
                            'pitcher_depth', 'pitcher_team_id', 'pitcher_team_nickname', 'home_team_name', 'home_team_market', 'home_team_abbr',
                            'away_team_name', 'away_team_market', 'away_team_abbr', 'stadium_name', 'stadium_market', 'stadium_surface', 'stadium_address',
                            'stadium_city', 'stadium_state', 'stadium_zip', 'stadium_country', 'stadium_field_orientation', 'stadium_time_zone', 'away_stadium_id',
                            'away_stadium_name', 'away_stadium_market', 'away_stadium_surface', 'away_stadium_address', 'away_stadium_city', 'away_stadium_state', 'away_stadium_zip',
                            'away_stadium_country', 'away_stadium_field_orientation', 'away_stadium_stadium_type', 'away_stadium_time_zone', 'Unnamed: 0_y'))

In [26]:
data = data.drop(columns=columns_to_remove_fp)

In [31]:
data.to_csv('../raw_data/final_raw_data.csv', index=True)